In [9]:
import tensorflow as tf
import numpy as np

#기존에 생성된 Graph 삭제, Reset
tf.reset_default_graph()

- Jupyter Notebook과 같은 Interactive Python 환경에서 Tensorflow와 Tensorboard를 사용하면, Jupyter Kernel을 재시작하지 않는 이상 변수들의 Context가 그대로 유지 되는 문제가 생긴다.

- Tensorboard를 사용하려 할 때, Interactive Python 환경에서 같은 코드를 두 번 이상 실행하면 Error가 발생한다. 그 이유는 하나의 Kernel에서 Tensorflow Graph를 만들어놓고, 그 Graph가 지워지지 않은 상태에서 다시 같은 Graph를 생성하면서 Error가 발생하는 것이다.

- tf.reset_default_graph() 메서드는 기존에 생성된 Graph를 모두 삭제하고 Reset 시켜 중복되는 것을 막아준다.

- 일반적인 Python 실행 환경에서는 큰 문제가 없지만, Context가 유지되는 Jupyter Notebook 환경에서 실행할 경우를 위해, 반드시 tf.reset_default_graph() 메서드를 통해 Reset 시켜주도록 하자

In [10]:
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)


with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph

    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))

    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.691562 [array([[ 0.05302263, -0.48696464],
       [-0.67189783, -0.85061324]], dtype=float32), array([[-1.04882157],
       [ 0.43824226]], dtype=float32)]
200 0.35014 [array([[ 6.23620939, -3.81494451],
       [-8.99192905, -8.74805164]], dtype=float32), array([[-7.18007469],
       [ 6.53715611]], dtype=float32)]
400 0.347782 [array([[  6.5674324 ,  -4.16113663],
       [-10.33017445, -10.1422472 ]], dtype=float32), array([[-8.28625011],
       [ 7.66059208]], dtype=float32)]
600 0.347205 [array([[  6.74903965,  -4.34823847],
       [-11.11951637, -10.96049595]], dtype=float32), array([[-8.93936062],
       [ 8.32681751]], dtype=float32)]
800 0.346967 [array([[  6.87450981,  -4.47612762],
       [-11.68875694, -11.54836559]], dtype=float32), array([[-9.41116428],
       [ 8.80879593]], dtype=float32)]
1000 0.346844 [array([[  6.97051907,  -4.57322645],
       [-12.1384058 , -12.01163197]], dtype=float32), array([[-9.78462791],
       [ 9.19058418]], dtype=float32)]
1200 0.346771 

9800 0.346576 [array([[  9.03981876,  -7.63067055],
       [-19.45006371, -19.42674637]], dtype=float32), array([[-13.35479069],
       [ 12.82136917]], dtype=float32)]
10000 0.346576 [array([[  9.12253094,  -7.79844332],
       [-19.57845497, -19.51748657]], dtype=float32), array([[-13.36922455],
       [ 12.83705425]], dtype=float32)]

Hypothesis:  [[  5.00005245e-01]
 [  9.99997258e-01]
 [  5.00004292e-01]
 [  3.59268961e-06]] 
Correct:  [[ 1.]
 [ 1.]
 [ 1.]
 [ 0.]] 
Accuracy:  0.75


- $tensorboard --logdir=./logs
- url : http://0.0.0.0:6006/